# Capstone Project - The Battle of the Neighborhoods

## Introduction

This project aim is to find the neighborhood places in Toronto. The places which are extracted from the data are hotels,parks,shopping malls,restaurants etc.This project will show the entertainments which are surrounded by a particular locations.


In [3]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
Image(url= "http://kimmemyles.com/wp-content/uploads/2014/09/Screen-Shot-2014-03-26-at-12.46.10-PM.png")

## Data Description

The data used in this project is provided by Foursquare location data. The data are grouped by landscape area, and each area included the information about this area and all information about restaurants, cafes, and stores which in this area.

# Import Libraries

In [4]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



## Define Foursquare Credentials

In [5]:
ClIENT_ID = 'T4QZ2ZUL4PGM5TKB0OMLMQ1QHZMNKSGNDZXWCS0X0NOW32ER' # your Foursquare ID
ClIENT_SECRET = 'IP5JEXXCNBDGQKYJDRRIAQDFBO3TPFEOMYPLSART2WQMS03Y' # your Foursquare Secret
VERSION = '20180604'
LIMIT =100

## Define the city and get its latitude & longitude

In [6]:
# define the city and get its latitude & longitude 
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


## Search for Hotels

In [7]:
# search for hotels
search_query = 'Hotel'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=T4QZ2ZUL4PGM5TKB0OMLMQ1QHZMNKSGNDZXWCS0X0NOW32ER&client_secret=IP5JEXXCNBDGQKYJDRRIAQDFBO3TPFEOMYPLSART2WQMS03Y&ll=43.6534817,-79.3839347&v=20180604&query=Hotel&radius=10000&limit=100'

In [8]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [10]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,51d212c3498ebf27dc469bc9,Chelsea Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592471111,False,33 Gerrard Street West,at Yonge St,43.658498,-79.383097,"[{'label': 'display', 'lat': 43.65849759157591...",562,M5G 1Z4,CA,Toronto,ON,Canada,"[33 Gerrard Street West (at Yonge St), Toronto...",NaN,NaN
1,4ab2d511f964a5209b6c20e3,Sheraton Centre Toronto Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592471111,False,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",324,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,NaN
2,4ad4c05bf964a520a3f520e3,Bond Place Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592471111,False,65 Dundas St E,at Bond St.,43.656188,-79.378452,"[{'label': 'display', 'lat': 43.65618805882607...",534,M5B 2G8,CA,Toronto,ON,Canada,"[65 Dundas St E (at Bond St.), Toronto ON M5B ...",NaN,NaN
3,4ae61cf6f964a520caa421e3,Pantages Hotel & Spa,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592471111,False,200 Victoria St,at Shuter St,43.654498,-79.379035,"[{'label': 'display', 'lat': 43.65449797039222...",410,NaN,CA,Toronto,ON,Canada,"[200 Victoria St (at Shuter St), Toronto ON, C...",NaN,NaN
4,4b68aed1f964a520de862be3,The Rex Hotel Jazz & Blues Bar,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",v-1592471111,False,194 Queen St W,Queen & St. Patrick,43.650505,-79.388577,"[{'label': 'display', 'lat': 43.65050475544005...",499,M5V 1Z1,CA,Toronto,ON,Canada,"[194 Queen St W (Queen & St. Patrick), Toronto...",62225795,NaN


In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [12]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Chelsea Hotel,Hotel,33 Gerrard Street West,at Yonge St,43.658498,-79.383097,"[{'label': 'display', 'lat': 43.65849759157591...",562,M5G 1Z4,CA,Toronto,ON,Canada,"[33 Gerrard Street West (at Yonge St), Toronto...",NaN,51d212c3498ebf27dc469bc9
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",324,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,4ab2d511f964a5209b6c20e3
2,Bond Place Hotel,Hotel,65 Dundas St E,at Bond St.,43.656188,-79.378452,"[{'label': 'display', 'lat': 43.65618805882607...",534,M5B 2G8,CA,Toronto,ON,Canada,"[65 Dundas St E (at Bond St.), Toronto ON M5B ...",NaN,4ad4c05bf964a520a3f520e3
3,Pantages Hotel & Spa,Hotel,200 Victoria St,at Shuter St,43.654498,-79.379035,"[{'label': 'display', 'lat': 43.65449797039222...",410,NaN,CA,Toronto,ON,Canada,"[200 Victoria St (at Shuter St), Toronto ON, C...",NaN,4ae61cf6f964a520caa421e3
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,Queen & St. Patrick,43.650505,-79.388577,"[{'label': 'display', 'lat': 43.65050475544005...",499,M5V 1Z1,CA,Toronto,ON,Canada,"[194 Queen St W (Queen & St. Patrick), Toronto...",NaN,4b68aed1f964a520de862be3


In [13]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,Bond Place Hotel,Hotel,65 Dundas St E,43.656188,-79.378452,M5B 2G8,ON
3,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
5,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON
6,The Omni King Edward Hotel,Hotel,37 King Street East,43.649191,-79.376006,M5C 1E9,ON
7,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
8,"The Saint James Hotel, Ascend Hotel Collection",Hotel,26 Gerrard Street East,43.659466,-79.380994,M5B 1G3,ON
9,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON


In [14]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any')
clean_dataframe3.reset_index(drop=True).head()

,name,categories,address,lat,lng,postalCode,state
0,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,Bond Place Hotel,Hotel,65 Dundas St E,43.656188,-79.378452,M5B 2G8,ON
3,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
4,The Omni King Edward Hotel,Hotel,37 King Street East,43.649191,-79.376006,M5C 1E9,ON


In [15]:

# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,Bond Place Hotel,Hotel,65 Dundas St E,43.656188,-79.378452,M5B 2G8,ON
6,The Omni King Edward Hotel,Hotel,37 King Street East,43.649191,-79.376006,M5C 1E9,ON
7,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
8,"The Saint James Hotel, Ascend Hotel Collection",Hotel,26 Gerrard Street East,43.659466,-79.380994,M5B 1G3,ON
9,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
10,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
11,Beverley Hotel,Hotel,335 Queen Street West,43.649701,-79.392114,M5V 2A4,ON
12,The Grand Hotel & Suites Toronto,Hotel,225 Jarvis St.,43.656449,-79.374110,M5B 2C1,ON


In [16]:
#delete rows which has duplicate hotel's name
df_hotels = clean_dataframe3.drop_duplicates(subset='name', keep="first")
df_hotels.head()

,name,categories,address,lat,lng,postalCode,state
0,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,Bond Place Hotel,Hotel,65 Dundas St E,43.656188,-79.378452,M5B 2G8,ON
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
6,The Omni King Edward Hotel,Hotel,37 King Street East,43.649191,-79.376006,M5C 1E9,ON


## search for Parks

In [17]:
# search for Parks
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=T4QZ2ZUL4PGM5TKB0OMLMQ1QHZMNKSGNDZXWCS0X0NOW32ER&client_secret=IP5JEXXCNBDGQKYJDRRIAQDFBO3TPFEOMYPLSART2WQMS03Y&ll=43.6534817,-79.3839347&v=20180604&query=Park&radius=10000&limit=100'

In [18]:
# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

In [19]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
park_dataframe = pd.json_normalize(venues)
park_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4b170e47f964a5208cc123e3,Queen's Park Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1592471331,False,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",NaN
1,4b54deadf964a520a3d027e3,Grange Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1592471331,False,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",663,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",NaN
2,4b9d206bf964a520e69136e3,Queen's Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1592471331,False,University Ave.,at Wellesley Ave.,43.663946,-79.392180,"[{'label': 'display', 'lat': 43.66394609897775...",1340,M5R 2E8,CA,Toronto,ON,Canada,"[University Ave. (at Wellesley Ave.), Toronto ...",NaN
3,4bb79860cf2fc9b616779e02,Berczy Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1592471331,False,35 Wellington St. East,NaN,43.648048,-79.375172,"[{'label': 'display', 'lat': 43.64804835388851...",929,NaN,CA,Toronto,ON,Canada,"[35 Wellington St. East, Toronto ON, Canada]",NaN
4,4b9177e4f964a520d2be33e3,TTC Streetcar #403 - Victoria park,"[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L...",v-1592471331,False,NaN,NaN,43.650735,-79.386921,"[{'label': 'display', 'lat': 43.650735, 'lng':...",389,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN


In [20]:
# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Queen's Park Subway Station,Metro Station,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",4b170e47f964a5208cc123e3
1,Grange Park,Park,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",663,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",4b54deadf964a520a3d027e3
2,Queen's Park,Park,University Ave.,at Wellesley Ave.,43.663946,-79.392180,"[{'label': 'display', 'lat': 43.66394609897775...",1340,M5R 2E8,CA,Toronto,ON,Canada,"[University Ave. (at Wellesley Ave.), Toronto ...",4b9d206bf964a520e69136e3
3,Berczy Park,Park,35 Wellington St. East,NaN,43.648048,-79.375172,"[{'label': 'display', 'lat': 43.64804835388851...",929,NaN,CA,Toronto,ON,Canada,"[35 Wellington St. East, Toronto ON, Canada]",4bb79860cf2fc9b616779e02
4,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,NaN,43.650735,-79.386921,"[{'label': 'display', 'lat': 43.650735, 'lng':...",389,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",4b9177e4f964a520d2be33e3


In [21]:
# delete unnecessary columns
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2.head()

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
1,Grange Park,Park,NaN,43.652488,-79.392053,NaN,ON
2,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
3,Berczy Park,Park,35 Wellington St. East,43.648048,-79.375172,NaN,ON
4,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,43.650735,-79.386921,NaN,ON


In [22]:
# delete rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any')
clean_park_dataframe3.reset_index(drop=True)

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
1,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
2,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
3,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
4,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
5,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
6,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
7,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
8,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
9,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON


In [23]:
# delete rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park.reset_index().head()

,index,name,categories,address,lat,lng,postalCode,state
0,2,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
1,9,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
2,14,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
3,16,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
4,23,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON


## Search for Restaurants

In [24]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=T4QZ2ZUL4PGM5TKB0OMLMQ1QHZMNKSGNDZXWCS0X0NOW32ER&client_secret=IP5JEXXCNBDGQKYJDRRIAQDFBO3TPFEOMYPLSART2WQMS03Y&ll=43.6534817,-79.3839347&v=20180604&query=Restaurant&radius=10000&limit=100'

In [25]:
# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

In [26]:
# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = pd.json_normalize(venues)
Restaurant_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1592471449,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,5750b013498e755287c6de97,Some Time BBQ Grill Restaurant 碳烤屋,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",v-1592471449,False,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,NaN
2,4ad4c05cf964a52006f620e3,Victoria's Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1592471449,False,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,498556908,NaN
3,4bd47e6fcfa7b7139f2924da,Studio Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1592471449,False,389 Church St.,43.661500,-79.379319,"[{'label': 'display', 'lat': 43.66150015906530...",966,M5B,CA,Toronto,ON,Canada,"[389 Church St. (at Carlton St.), Toronto ON M...",at Carlton St.,NaN,NaN
4,4ae29812f964a520288f21e3,Swatow Restaurant 汕頭小食家,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1592471449,False,309 Spadina Ave.,43.653866,-79.398334,"[{'label': 'display', 'lat': 43.65386562507761...",1160,M5T 2E6,CA,Toronto,ON,Canada,[309 Spadina Ave. (btwn Dundas St. W & D'Arcy ...,btwn Dundas St. W & D'Arcy St.,NaN,NaN


In [27]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]


# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,5750b013498e755287c6de97
2,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,NaN,4ad4c05cf964a52006f620e3
3,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,"[{'label': 'display', 'lat': 43.66150015906530...",966,M5B,CA,Toronto,ON,Canada,"[389 Church St. (at Carlton St.), Toronto ON M...",at Carlton St.,NaN,4bd47e6fcfa7b7139f2924da
4,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,"[{'label': 'display', 'lat': 43.65386562507761...",1160,M5T 2E6,CA,Toronto,ON,Canada,[309 Spadina Ave. (btwn Dundas St. W & D'Arcy ...,btwn Dundas St. W & D'Arcy St.,NaN,4ae29812f964a520288f21e3


In [29]:
# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2.head()


,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,NaN,ON
2,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
3,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON
4,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON


In [30]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
2,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
3,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON
4,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
6,Green Tea Restaurant Downtown,Chinese Restaurant,261 Spadina Avenue. Upper level,43.652488,-79.397501,M5T 2E3,ON
7,Cottage Restaurant & Lounge,Thai Restaurant,338 Jarvis St.,43.662770,-79.376894,M4Y 2G6,ON
8,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
9,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
10,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,M5V 1J5,ON
11,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON


## Search for Cafeteria

In [31]:
# search for Cafeteria
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=T4QZ2ZUL4PGM5TKB0OMLMQ1QHZMNKSGNDZXWCS0X0NOW32ER&client_secret=IP5JEXXCNBDGQKYJDRRIAQDFBO3TPFEOMYPLSART2WQMS03Y&ll=43.6534817,-79.3839347&v=20180604&query=Cafeteria&radius=10000&limit=100'

In [32]:
# Send the GET Request and examine the results
cresults = requests.get(url).json()
#cresults

In [34]:
# assign relevant part of JSON to venues
venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe =pd.json_normalize(venues)
Cafeteria_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,4f9165bde4b04fef0087e4e2,Cafeteria,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1592471538,False,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",448,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN
1,528e99a211d262edc3708a6f,Stay Cafeteria 慢走,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1592471538,False,388 Spadina Ave,43.655454,-79.399163,"[{'label': 'display', 'lat': 43.65545427446646...",1246,CA,Toronto,ON,Canada,"[388 Spadina Ave (at Nassau St), Toronto ON M5...",at Nassau St,M5T 2G5
2,4b57256bf964a520332828e3,Sears Cafeteria,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1592471538,False,Yonge,43.656038,-79.380672,"[{'label': 'display', 'lat': 43.65603785909391...",387,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON, Canada]",Dundas,NaN
3,53bd6dfe498e6c1d3bc9333e,Industrial Alliance Cafeteria,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1592471538,False,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",505,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN
4,524f1c8f498e983dde61ada0,Chestnut Tree Cafeteria,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1592471538,False,89 Chestnut Street,43.648231,-79.384045,"[{'label': 'display', 'lat': 43.648231, 'lng':...",584,CA,Toronto,ON,Canada,"[89 Chestnut Street, Toronto ON, Canada]",NaN,NaN


In [35]:
# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()


,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",448,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN,4f9165bde4b04fef0087e4e2
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,"[{'label': 'display', 'lat': 43.65545427446646...",1246,CA,Toronto,ON,Canada,"[388 Spadina Ave (at Nassau St), Toronto ON M5...",at Nassau St,M5T 2G5,528e99a211d262edc3708a6f
2,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,"[{'label': 'display', 'lat': 43.65603785909391...",387,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON, Canada]",Dundas,NaN,4b57256bf964a520332828e3
3,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",505,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,53bd6dfe498e6c1d3bc9333e
4,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,"[{'label': 'display', 'lat': 43.648231, 'lng':...",584,CA,Toronto,ON,Canada,"[89 Chestnut Street, Toronto ON, Canada]",NaN,NaN,524f1c8f498e983dde61ada0


In [37]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2.head()

,name,categories,address,lat,lng,state,postalCode
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,ON,NaN
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,ON,M5T 2G5
2,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,ON,NaN
3,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,ON,NaN
4,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,ON,NaN


In [39]:
# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any')
df_Cafeteria.reset_index()

,index,name,categories,address,lat,lng,state,postalCode
0,1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,ON,M5T 2G5
1,9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,ON,M4V 1H7
2,19,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,ON,M5B 1W8
3,21,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,ON,M5B 2K3
4,26,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,ON,m3c 3k7
5,27,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,ON,M5S 1J5
6,30,Eurest Dishes Cafe,Food Court,100 Wynford Drive,43.727279,-79.332272,ON,M3C 4B4
7,33,Upstairs Cafe,Corporate Cafeteria,26 Prince Andrew Place,43.725383,-79.345481,ON,M3C 2H4


## Search for Shopping Stores

In [40]:
# search for Shopping
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=T4QZ2ZUL4PGM5TKB0OMLMQ1QHZMNKSGNDZXWCS0X0NOW32ER&client_secret=IP5JEXXCNBDGQKYJDRRIAQDFBO3TPFEOMYPLSART2WQMS03Y&ll=43.6534817,-79.3839347&v=20180604&query=Shopping&radius=1000&limit=100'

In [41]:
# Send the GET Request and examine the results
sresults = requests.get(url).json()
#sresults

In [42]:
# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = pd.json_normalize(venues)
Shopping_dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,5840cafe06f1a34af80cc609,Saks Fifth Avenue Club - Personal Shopping,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",v-1592471616,False,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",424,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN
1,4b9e86dff964a520e6eb36e3,TD Centre Shopping Concourse,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1592471616,False,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",741,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN
2,4ad77a12f964a520260b21e3,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1592471616,False,220 Yonge St,btwn Queen & Dundas,43.654540,-79.380677,NaN,287,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON"


In [43]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",424,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN,5840cafe06f1a34af80cc609
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",741,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN,4b9e86dff964a520e6eb36e3
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,btwn Queen & Dundas,43.654540,-79.380677,NaN,287,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON",4ad77a12f964a520260b21e3


In [44]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON


In [45]:
# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping

,name,categories,address,lat,lng,postalCode,state
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON


## Generate maps to visualize venues and how they cluster together

In [56]:
# create dataframe of hotels, shopping stores and Cafeteria
neighbourhood_df = pd.concat([df_hotels, df_Cafeteria, df_Shopping, df_Restaurant,df_park], ignore_index=True)
neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,Bond Place Hotel,Hotel,65 Dundas St E,43.656188,-79.378452,M5B 2G8,ON
3,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
4,The Omni King Edward Hotel,Hotel,37 King Street East,43.649191,-79.376006,M5C 1E9,ON
...,...,...,...,...,...,...,...
79,Coronation Park,Park,711 Lake Shore Blvd. W,43.634553,-79.405403,M5V 3T7,ON
80,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
81,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON
82,Sir Winston Churchill Park,Park,301 St Clair Ave W,43.683732,-79.409881,M4V 1S4,ON


In [83]:
# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
neighbourhood_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(neighbourhood_df['lat'], neighbourhood_df['lng'], 
                                           neighbourhood_df['name'], neighbourhood_df['categories'],neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        key_on = categories,
        threshold_scale=[0,1,2,3],
        fill_color= 'lightblue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map